# Setting up the initial simulation grid


### Simulation parameters $H$

From each simulation we run, we will output a number of snapshots, of order 10. 

Moreover, for each snapshot we extract different sets of simulated Lyman-$\alpha$ skewers (normalized quasar spectra), after applying different rescalings of the temperatures in the snapshot. 
These skewers are written to disk.

Finally, from each set of skewers we measure different 1D power spectra, after rescaling the mean optical depth in the spectra. 
This optical depth rescaling is trivial, and it is done on the fly. 
Each of these measured power spectra is fed to the emulator. 


Let's discuss the parameterization of each of the simulation packages:

- As we discuss above, the emulator labels the measured power with the set of parameters $ M = \{ M_P, f, \lambda_F, \bar F, \sigma_T, \gamma \} $.

- Each set of simulated skewers, written to disk, is described by a subset of these $ \{ M_P, f, \lambda_F, \sigma_T, \gamma \} $, since we will have different values of $\bar F$ from the skewers.

- Each snapshot is described by an even smaller subset of parameters, $ \{ M_P, f, \lambda_F \} $, since we will reprocess the snapshot for different temperature-density relations.

If we assumed that we can do as much rescaling of mean flux and temperature as we wanted, then this last set of parameters $ \{ M_P, f, \lambda_F \} $ would be the only relevant ones. If we can not rescale as much as we would like to (because the rescaling breaks down at some point?), then we would need to label the snapshots with a "central temperature" and "central mean flux", around which we would perturb. 

Now, since we are describing the linear power in comoving units, the shape of the linear power will be the same in all snapshots. The only parameters that will vary are the amplitude of the linear power, the logarithmic growth rate $f$ and the filtering length $\lambda_F$. So each simulation will be described by two shape parameters, and $N_z \times \{ A_p, f, \lambda_F \}$, where $N_z$ is the number of snapshots. 

$$ H = \{ n_p, \alpha_p, N_z \times \{ A_p, f, \lambda_F \} \} $$


The question is: how do we decide what simulation to run, i.e., what configuration files to use, if we are handed a set of parameters $H$? We will not aim at having a perfect match between the two, we just want to get on the right ball park.

From the point of view of cosmological parameters:
 - We start by defining a fiducial cosmology, somewhere around Planck, but WITHOUT NEUTRINOS. This is important, since our simulations will not have neutrinos. All quantities with $^0$ will refer to that fiducial cosmology. 
 - In flat $\Lambda$CDM models, we can translate the requirement of a given $f_\star$ into a requirement of a given $\Omega_\star = \Omega_{cb}(z_\star)$, using $f_\star = \Omega_\star^{0.55}$.
 - We can then translate the requirement on $\Omega_\star$ to a requirement on $\Omega_{cb} = \Omega_{cb}(z=0)$, using $\Omega_\star = \Omega_{cb} (1+z_\star)^3 / ( \Omega_{cb} (1+z_\star)^3 + 1 - \Omega_{cb} ) $. 
 - We will keep fixed the values of $\Omega_c h^2$ and $\Omega_b h^2$, so the requirement on $\Omega_{cb}$ can be related to a requirement on $h$, using $h^2 = (\Omega_b h^2 + \Omega_c h^2)/\Omega_{cb} $.
 - We will also measure the shape of the linear power at $z_\star$ around $k_p$, and modify the slope and running of the primordial power to match the required shape.
 - So the cosmology in the simulation will be specified by $\{ \Omega_\star, A_p, n_p, \alpha_p \}$, where $A_p$ will be computed at $z_\star$.

From the point of view of the pressure smoothing:
 - We will look at the value of the filtering length at a pre-HeII and at a post-HeII redshift, say z=2 and z=4. 
 - We will use the Oñorbe models to translate the required values for $\lambda_F$ pre- and post-HeII reionization to a required thermal history. It will probably be enough to specify a scaling of the heating rates before ($\mu_H$) and after ($\mu_{He}$) He_II reionizaion. 

### Initial simulation grid

We have 6 simulation parameters: 

$$ H = \{ \mu_H, \mu_{He}, \Omega_\star, \Delta_p^2, n_p, \alpha_p \} $$

We will choose the following parameter volume for the simulations:
 - $ 0.5 < \mu_H < 2.0 $
 - $ 0.5 < \mu_{He} < 2.0 $
 - $ 0.950 < \Omega_\star < 0.975 $
 - $ 0.25 < \Delta_p^2 < 0.45 $
 - $ -2.4 < n_p < -2.3 $
 - $ -0.12 < \alpha_p < -0.08 $

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import copy
## Set default plot size, as normally its a bit too small
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 120
mpl.rcParams['figure.dpi'] = 120
import fit_pk
import read_genic
import write_config
import camb_cosmo
import latin_hypercube

In [2]:
def get_sim_params(z_star=3.0,kp_Mpc=0.7,add_running=False,add_mu_H=False):
    """Return list of simulation parameters, with name and allowed range"""
    params={}
    params['Om_star']={'ip':len(params), 'min_val':0.950, 'max_val':0.975, 'z_star':z_star}
    params['Delta2_star']={'ip':len(params), 'min_val':0.25, 'max_val':0.45, 'z_star':z_star, 'kp_Mpc':kp_Mpc}
    params['n_star']={'ip':len(params), 'min_val':-2.4, 'max_val':-2.3, 'z_star':z_star, 'kp_Mpc':kp_Mpc}
    if add_running:
        params['alpha_star']={'ip':len(params), 'min_val':-0.12, 'max_val':-0.08, 'z_star':z_star, 'kp_Mpc':kp_Mpc}
    params['mu_He']={'ip':len(params), 'min_val':0.5, 'max_val':2.0}
    if add_mu_H:
        params['mu_H']={'ip':len(params), 'min_val':0.5, 'max_val':2.0}
 
    return params

- Setup Latin hypercube

In [3]:
# get list of simulation parameters
z_star=3.0
kp_Mpc=0.7
param_space = get_sim_params(z_star=z_star,kp_Mpc=kp_Mpc,add_running=True)
Npar=len(param_space)
param_limits=np.empty([Npar,2])
for key,param in param_space.items():
    ip=param['ip']
    #print("{} = {}".format(key, ip))
    param_limits[ip][0]=param['min_val']
    param_limits[ip][1]=param['max_val']

In [4]:
# generate Latin hypercube 
nsamples=5
seed=123
cube=latin_hypercube.get_hypercube_samples(param_limits, nsamples, prior_points = None, seed=seed)

 - Setup fiducial cosmology

In [5]:
cosmo_fid = camb_cosmo.get_cosmology()
camb_cosmo.print_info(cosmo_fid)
# get linear power parameters, in comoving units
linP_params_fid=fit_pk.parameterize_cosmology_Mpc(cosmo_fid,z_star=z_star,kp_Mpc=kp_Mpc)
print('fiducial cosmology parameters',linP_params_fid)

H0 = 67.0 ; Omega_b h^2 = 0.022 ; Omega_c h^2 = 0.12 ; Omega_k = 0.0 ; ommnuh2 = 0.0 ; T_CMB = 2.7255 ; A_s = 2.1e-09 ; n_s = 0.96 ; alpha_s = 0.0
fiducial cosmology parameters {'f_star': 0.98136698840516934, 'g_star': 0.9677508579459803, 'linP_Mpc': poly1d([-0.10748482, -2.30758676,  3.02860938])}


 - For each point in cube, write configuration files to run simulation

In [9]:
sim_dirs='test_cube'
write_config.mkdir_if_not_exists(sim_dirs)
for sample in range(nsamples):
    sim_params=cube[sample]
    print(sample,sim_params)
    cosmo_sim=fit_pk.cosmo_from_sim_params(param_space,sim_params,cosmo_fid,linP_params_fid,z_star,kp_Mpc,verbose=False)
    sim_dir=sim_dirs+'/sim_'+str(sample)+'/'
    write_config.mkdir_if_not_exists(sim_dir)
    file_name=sim_dir+'paramfile'
    # write GenIC and MP-Gadget parameters, for both simulations in pair
    for paired in [False,True]:
        write_config.write_genic_file(file_name,cosmo_sim,paired=paired)
        write_config.write_gadget_file(file_name,cosmo_sim,paired=paired)
    # compute fit parameters and store in JSON format
    linP_params_sim=fit_pk.parameterize_cosmology_Mpc(cosmo_sim,z_star=z_star,kp_Mpc=kp_Mpc)  
    write_config.write_json_file(file_name,param_space,sim_params,linP_params_sim)

0 [ 0.9625  0.39   -2.37   -0.1     1.85  ]
Modify GenIC to allow for running 0.007490847585157112
Modify GenIC to allow for running 0.007490847585157112
1 [ 0.9575  0.31   -2.35   -0.092   0.65  ]
Modify GenIC to allow for running 0.015447782137315694
Modify GenIC to allow for running 0.015447782137315694
2 [ 0.9675  0.35   -2.33   -0.116   0.95  ]
Modify GenIC to allow for running -0.008487302913790226
Modify GenIC to allow for running -0.008487302913790226
3 [ 0.9525  0.27   -2.39   -0.108   1.25  ]
Modify GenIC to allow for running -0.0005401485549182267
Modify GenIC to allow for running -0.0005401485549182267
4 [ 0.9725  0.43   -2.31   -0.084   1.55  ]
Modify GenIC to allow for running 0.023440500423058286
Modify GenIC to allow for running 0.023440500423058286
